### Import the necessary program libraries

In [5]:
import pandas as pd
from classes import KMeansClusterMain, BaseLoader
import plotly.express as px
from sqlalchemy import create_engine
from settings import mariadb_string, mariadb_config
from funcs import get_density
from pathlib import Path

### Automate of receiving pics

In [6]:

loader = BaseLoader(mariadb_config)
symbols = loader.select("select symbol from indexes;")

data_select_query = """select 
	*
from
	stock_price s
where
	s.symbol = '{}'
group BY
	s.`date`
order by
	s.`date` asc;"""

for item in symbols:
    df_stock = pd.read_sql(data_select_query.format(item[0]), create_engine(mariadb_string))
    df_stock["date"] = pd.to_datetime(df_stock["date"])
    
    # Compute the percentage change
    df_stock["perc_change"] = df_stock["close"].pct_change()
    df_stock.dropna(inplace=True)
    df_stock = df_stock[df_stock['perc_change'] != 0.0]
    
    # Here comes KMeans
    datapoints = df_stock["perc_change"].to_list()
    kmeans = KMeansClusterMain(3)
    kmeans.fit(datapoints) 
    
    # Assign the clusters
    df_stock["cluster"] = kmeans.labels
    df_stock["cluster"] = pd.to_numeric(df_stock["cluster"])
    
    # Get the density for the colormap
    df_stock["density"] = df_stock["cluster"].rolling(21).apply(get_density, kwargs={'orig_density': kmeans.centroids})
    
    # cluster_str needed for kmeans volatility cluster graph.
    df_stock["cluster_str"] = df_stock.cluster.astype(str)
    df_stock['perc_change'] = df_stock['perc_change'] * 100
    
    # Calculate the density
    describe_ser = df_stock["density"].describe()
    upper_limit = describe_ser["mean"] + (describe_ser["std"] * 2)
    lower_limit = describe_ser["mean"] - (describe_ser["std"] * 2)
    
    # Show the density
    fig = px.line(df_stock, x="date", y="density", title="Stock on Daily Basis",
                      labels={"date": "Date", "density": "Cluster Aufsummierung"})
    
    fig.add_hline(y=upper_limit, line_dash="dash", line_color="red", annotation_text=f"Grenze bei {round(upper_limit,4)}")
    fig.add_hline(y=lower_limit, line_dash="dash", line_color="red", annotation_text=f"Grenze bei {round(lower_limit,4)}")
    
    
    fig.write_image(str(Path.cwd().joinpath("images", item[0] + "_kmeans_analyse")) + ".png")
    print(f"Image for {item[0]} saved!")

count    1329.000000
mean        0.019661
std         0.078821
min        -0.188361
25%        -0.042094
50%         0.016606
75%         0.073373
max         0.223507
Name: density, dtype: float64
count    1343.000000
mean        0.023692
std         0.085133
min        -0.271408
25%        -0.032490
50%         0.025091
75%         0.082672
max         0.291003
Name: density, dtype: float64
count    739.000000
mean       0.044025
std        0.105590
min       -0.319073
25%       -0.005265
50%        0.036854
75%        0.113626
max        0.312276
Name: density, dtype: float64
count    1298.000000
mean        0.011310
std         0.057319
min        -0.122133
25%        -0.035686
50%         0.016517
75%         0.055385
max         0.229418
Name: density, dtype: float64
count    1347.000000
mean        0.014874
std         0.073013
min        -0.228689
25%        -0.032810
50%         0.018179
75%         0.069167
max         0.241820
Name: density, dtype: float64
count    1294.0000